In [1]:
import mysql.connector
import csv
import logging
import pandas as pd

In [2]:
conn1 = mysql.connector.connect(host='localhost', user='nhannguyen', password='123123', database='staging', autocommit=True)
print(conn1)
cur1 = conn1.cursor()

conn2 = mysql.connector.connect(host='localhost', user='nhannguyen', password='123123', database='data_warehouse', autocommit=True)
print(conn2)
cur2 = conn2.cursor()

In [7]:
cur.callproc('insert_data',["Hà Giang","2022-09-26","07:42:00",25,"Bầu trời quang đãng",81,10.0,0.9,0,"Rất kém"])

('Hà Giang',
 '2022-09-26',
 '07:42:00',
 25,
 'Bầu trời quang đãng',
 81,
 Decimal('10.0'),
 Decimal('0.9'),
 0,
 'Rất kém')

In [ ]:
# logging config
logging.basicConfig(level=logging.INFO, filename='log_file.log', format='%(asctime)s - %(levelname)s - %(message)s')

In [33]:
# create database staging
staging = 'create database staging'
# create table weather_data
weather_data = 'create table weather_data ( \
   ID int primary key auto_increment,\
   Name varchar(20), \
   Date date, \
   Time time, \
   Temperature tinyint, \
   Description nvarchar(50), \
   Humidity tinyint, \
   Vision tinyint, \
   Wind float, \
   UV_index float, \
   Air_quality varchar(10) \
    )'
insert='insert into weather_data (Name,Date, Time,Temperature,Description,Humidity,Vision,Wind,UV_index,Air_quality) \
   values \
   ("Hà Giang","2022-09-26","07:42:00",25,"Bầu trời quang đãng",81,10.0,0.9,0,"Rất kém"), \
   ("Cao Bằng","2022-09-26","07:42:00",26,"Bầu trời quang đãng",81,10.0,0.96,0,"Rất kém")'
select='select * from weather_data'

In [15]:
# create table staging
cur.execute(staging)

In [29]:
# drop table weather_data
cur.execute('drop table if exists weather_data')

In [32]:
# create table weather_data
cur.execute(weather_data)

In [35]:
# insert values into table weather_data
cur.execute(insert)
# conn.commit()

In [50]:
csv_file='etl_data.csv'
csv_insert='insert into weather_data (Name,Date, Time,Temperature,Description,Humidity,Vision,Wind,UV_index,Air_quality) \
   values \
   (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
select='select * from weather_data'
with open(csv_file, 'r', encoding='utf-8') as csvfile:
    csv_data=csv.reader(csvfile)
    for row in csv_data:
        cur.execute(csv_insert, row)

In [58]:
# delete all values from weather_data
cur.execute('truncate table weather_data')

In [4]:
try:
    # cur.execute('use demo;')
    cur.execute('select * from staging.staging_raw')
    result = cur.fetchall()
    print('success')
except:
    print('fail')
for x in cur:
    print(x)

success


In [8]:
cur1.execute('select * from staging.staging_dim_province')
result1 = cur1.fetchall()
df1 = pd.DataFrame(result1)
df1

,0,1,2,3
0,1,Hà Bắc,2022-11-06,0 days 09:25:00
1,2,Hà Nam,2022-11-06,0 days 09:26:00
2,3,Bắc Kạn,2022-11-06,0 days 09:25:00
3,4,Tuyên Quang,2022-11-06,0 days 09:25:00
4,5,Thái Nguyên,2022-11-06,0 days 09:25:00
...,...,...,...,...
58,59,Cần Thơ,2022-11-06,0 days 09:25:00
59,60,Hậu Giang,2022-11-06,0 days 09:25:00
60,61,Sóc Trăng,2022-11-06,0 days 09:26:00
61,62,Bạc Liêu,2022-11-06,0 days 09:26:00


In [9]:
cur2.execute('select id_province, name, start_date, expired_time from data_warehouse.dim_province')
result2 = cur2.fetchall()
df2 = pd.DataFrame(result2)
df2

,0,1,2,3
0,1,Hà Giang,2022-11-06,0 days
1,2,Cao Bằng,2022-11-06,0 days
2,3,Bắc Kạn,2022-11-06,0 days
3,4,Tuyên Quang,2022-11-06,0 days
4,5,Thái Nguyên,2022-11-06,0 days
...,...,...,...,...
58,59,Cần Thơ,2022-11-06,0 days
59,60,Hậu Giang,2022-11-06,0 days
60,61,Sóc Trăng,2022-11-06,0 days
61,62,Bạc Liêu,2022-11-06,0 days


In [10]:
for i in df1[0]:
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


### Close connection

In [63]:
cur.close()
conn.close()